In [3]:
'''
Lab | Random Forests

For this lab, you will be using the CSV files provided in the `files_for_lab` folder.

Instructions

- Apply the Random Forest algorithm to predict the `TARGET_B`. Please note that this column suffers from class imbalance. 
Fix the class imbalance using upsampling.
- Discuss the model predictions and it's impact in the bussiness scenario. Is the cost of a false positive equals to the cost of 
the false negative? How much the money the company will not earn because of missclassifications made by the model?
- Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. 
Which one? 
'''

"\nLab | Random Forests\n\nFor this lab, you will be using the CSV files provided in the `files_for_lab` folder.\n\nInstructions\n\n- Apply the Random Forest algorithm to predict the `TARGET_B`. Please note that this column suffers from class imbalance. \nFix the class imbalance using upsampling.\n- Discuss the model predictions and it's impact in the bussiness scenario. Is the cost of a false positive equals to the cost of \nthe false negative? How much the money the company will not earn because of missclassifications made by the model?\n- Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. \nWhich one?  Please checkout\n"

In [65]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.utils import resample
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import make_scorer, f1_score


categorical= pd.read_csv('categorical.csv')
display(categorical)


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,27,H,M,3,L,G,C,C,2,...,0,2,96,2,96,2,96,2,96,2
95408,TX,24,H,M,3,L,F,A,C,1,...,50,1,96,3,96,3,96,3,96,3
95409,MI,30,H,M,3,L,E,B,C,3,...,38,1,96,3,95,1,96,10,94,10
95410,CA,24,H,F,2,L,F,A,C,1,...,40,5,90,11,96,8,97,1,86,12


In [5]:
numerical= pd.read_csv('numerical.csv')
display(numerical)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,0,25.0,25.0,25.0,9,25.000000,184568,0,1,12
95408,1,48.000000,7,9,1,0,31,43,19,4,...,0,20.0,20.0,20.0,9,20.000000,122706,1,1,2
95409,1,60.000000,5,9,0,0,18,46,20,7,...,4,3.0,10.0,10.0,3,8.285714,189641,1,3,34
95410,0,58.000000,7,9,0,0,28,35,20,9,...,18,5.0,21.0,18.0,4,12.146341,4693,1,4,11


In [6]:
target_data= pd.read_csv('target.csv')
display(target_data)

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
95407,0,0.0
95408,0,0.0
95409,0,0.0
95410,1,18.0


In [7]:
class_distribution = target_data['TARGET_B'].value_counts()
print(class_distribution)

TARGET_B
0    90569
1     4843
Name: count, dtype: int64


In [8]:
majority_class = target_data[target_data['TARGET_B'] == 0]
minority_class = target_data[target_data['TARGET_B'] == 1]

minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

target_data_upsampled = pd.concat([majority_class, minority_upsampled])

In [9]:
class_distribution = target_data_upsampled['TARGET_B'].value_counts()
print(class_distribution)

TARGET_B
0    90569
1    90569
Name: count, dtype: int64


In [21]:
display(target_data_upsampled)

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
181133,1,10.0
181134,1,20.0
181135,1,15.0
181136,1,1.0


In [24]:
target_data_downsampled = resample(target_data_upsampled[target_data_upsampled['TARGET_B'] == 0], replace=True,n_samples=len(numerical),
                                   random_state=42)
display(target_data_downsampled)

,TARGET_B,TARGET_D
15795,0,0.0
860,0,0.0
76820,0,0.0
54886,0,0.0
6265,0,0.0
...,...,...
80899,0,0.0
86066,0,0.0
56613,0,0.0
37426,0,0.0


In [25]:
class_distribution1 = target_data_downsampled['TARGET_B'].value_counts()
print(class_distribution)

TARGET_B
0    90569
1    90569
Name: count, dtype: int64


In [11]:
columns_to_append= categorical[['HOMEOWNR', 'GENDER']]
columns_to_append = pd.DataFrame(columns_to_append)

In [12]:
numerical = pd.concat([numerical, columns_to_append], axis=1)

In [28]:
numerical.describe

<bound method NDFrame.describe of        TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
0          0  60.000000       5        9    0         0       39        34   
1          1  46.000000       6        9   16         0       15        55   
2          1  61.611649       3        1    2         0       20        29   
3          0  70.000000       1        4    2         0       23        14   
4          0  78.000000       3        2   60         1       28         9   
...      ...        ...     ...      ...  ...       ...      ...       ...   
95407      1  61.611649       5        9    0        14       36        47   
95408      1  48.000000       7        9    1         0       31        43   
95409      1  60.000000       5        9    0         0       18        46   
95410      0  58.000000       7        9    0         0       28        35   
95411      2  80.000000       5        8    3         1       33        22   

       WWIIVETS  LOCALGOV  ..

In [32]:
print(numerical['HOMEOWNR'].value_counts())

HOMEOWNR
H    74582
U    20830
Name: count, dtype: int64


In [46]:
print (numerical['GENDER'].value_counts())

GENDER
0        54234
1        39094
other     2084
Name: count, dtype: int64


In [47]:
mode_gender = numerical['GENDER'].mode()[0]  
numerical['GENDER'] = numerical['GENDER'].replace({'M': 1, 'F': 0, 'other': mode_gender})

HOMEOWNR
1    74582
0    20830
Name: count, dtype: int64


In [48]:
print(numerical['GENDER'].value_counts())

GENDER
0    56318
1    39094
Name: count, dtype: int64


In [41]:
numerical['HOMEOWNR'] = numerical['HOMEOWNR'].str.strip().replace({'H': 1, 'U': 0})
print(numerical['HOMEOWNR'].value_counts())

HOMEOWNR
1    74582
0    20830
Name: count, dtype: int64


In [51]:
numerical = numerical.reset_index(drop=True)
target_data_upsampled = target_data_upsampled.reset_index(drop=True)

In [52]:
print(numerical.duplicated().sum())
print(target_data_upsampled.duplicated().sum())

0
181067


In [53]:
X = numerical
y = target_data_downsampled['TARGET_B']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [58]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [ ]:
#So 

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_depth': [None, 2, 3],
    'max_features': ['sqrt']
}

rf = RandomForestRegressor(random_state=100)
grid_search = GridSearchCV(rf, param_grid, cv=5, return_train_score=True, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print("Best hyperparameters:", grid_search.best_params_)
rf_best = RandomForestRegressor(**grid_search.best_params_)
cross_val_scores = cross_val_score(rf_best, X_train, y_train, cv=10)
print("The mean R2 over the folds was {:.2f}".format(np.mean(cross_val_scores)))
rf_best.fit(X_train, y_train)
print("The R2 for the model on the TEST set is {:.3f}".format(rf_best.score(X_test, y_test)))
y_pred1 = rf_best.predict(X_test)
accuracy1 = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
The mean R2 over the folds was 1.00
The R2 for the model on the TEST set is 1.000
Accuracy: 1.0


In [ ]:
'''So the model is definitely overfitted, which I assume is because of the upsampling.
In this case, in which the company would have to spend money reaching out to every client, reaching to a client that haven't donated, assuming
that he or she has (so a false positive) would be more detrimental, specially considering that the number of clients that didn't donate
was quite high, and much bigger than the number of people that donated.

Assuming that the cost per client of the campaing was only $1, a really low value for real life standard, applying this model could cost the
company over $90,000
'''

In [69]:

rf2 = RandomForestClassifier(random_state=100)

# Define the parameter grid
param_grid2 = {
    'n_estimators': [50, 100, 150],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2']
}

# Create a scorer based on F1 score
f1_scorer = make_scorer(f1_score)

# Create the GridSearchCV object with the F1 score as the scoring metric
grid_search2 = GridSearchCV(rf, param_grid2, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=2)

# Fit the model with different hyperparameter combinations
grid_search2.fit(X_train, y_train)

# Display the best hyperparameters
print("Best hyperparameters:", grid_search2.best_params_)

# Create a RandomForestClassifier with the best hyperparameters
rf_best2 = RandomForestClassifier(class_weight='balanced', **grid_search.best_params_)

# Fit the model on the training set
rf_best2.fit(X_train, y_train)

# Evaluate the model on the test set using the F1 score
f1_score_test = f1_score(y_test, rf_best.predict(X_test))
print("F1 Score on the test set:", f1_score_test)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
F1 Score on the test set: 0.0


C:\Users\jorge\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [67]:
y_pred = rf_best.predict(X_test)
print("Predicted Class Distribution:", pd.Series(y_pred).value_counts())

Predicted Class Distribution: 0.0    19083
Name: count, dtype: int64


In [ ]:
''' So this model I created is completely useless, it seems. Could you please help me understand why this is happening? I imagine is because
of how my dataset is structured. Perhaps because I used 2 different dataframes?